In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("sales_data 3.csv")
df.head()

,Code,Employee Name,Group,Item Code,Item Group Name,Category,Brand,Item Name,Qty,Price,Total,I_Group
0,S-01-12-2023-274341,Zahid - 217,B.City,5616,Power & Smart Tech,Power Adapter,Anker,Anker PowerPort III 20W Cube,1,1150,1150.0,Power Tech
1,S-01-12-2023-274341,Zahid - 217,B.City,912,Apple Accessories,Data Cable,Apple,Apple USB-C to Lightning Cable 1m - Non Author...,1,1000,1000.0,Power Tech
2,S-01-12-2023-274341,Zahid - 217,B.City,22672,Used Devices,iPhone - Used,Apple,iPhone 11 128GB Green e-Sim Slim Box (Used),1,47500,47500.0,Devices
3,S-01-12-2023-274340,Arif - 007,B.City,19858,Power & Smart Tech,Power Adapter,Anker,Anker 511 Charger Nano 3 - 30W - White,1,1650,1650.0,Power Tech
4,S-01-12-2023-274339,Mehedi - 165,B.City,3037,Outfits,Macbook Cover,WiWu,WiWU Keyboard Protector for MacBook - Air 13.3...,1,850,850.0,N|A


In [5]:
def data_summary(df):
    print("Shape:", df.shape)
    print("\nData Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nDuplicated Rows:", df.duplicated().sum())
    print("\nDescriptive Stats:\n", df.describe(include='all'))

data_summary(df)


Shape: (421, 12)

Data Types:
 Code                object
Employee Name       object
Group               object
Item Code            int64
Item Group Name     object
Category            object
Brand               object
Item Name           object
Qty                  int64
Price                int64
Total              float64
I_Group             object
dtype: object

Missing Values:
 Code                0
Employee Name       0
Group               0
Item Code           0
Item Group Name     0
Category            0
Brand              36
Item Name           0
Qty                 0
Price               0
Total               0
I_Group             0
dtype: int64

Duplicated Rows: 0

Descriptive Stats:
                        Code Employee Name   Group     Item Code  \
count                   421           421     421    421.000000   
unique                  235            31       6           NaN   
top     S-01-12-2023-274252  Shawon - 031  B.City           NaN   
freq                      6

In [6]:
def handle_missing(df, strategy='mean'):
    df_clean = df.copy()
    for col in df_clean.columns:
        if df_clean[col].isnull().sum() > 0:
            if df_clean[col].dtype in ['int64', 'float64']:
                if strategy == 'mean':
                    df_clean[col].fillna(df_clean[col].mean(), inplace=True)
                elif strategy == 'median':
                    df_clean[col].fillna(df_clean[col].median(), inplace=True)
            else:
                df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
    return df_clean

def remove_duplicates(df):
    return df.drop_duplicates()

def clean_text_columns(df):
    df_clean = df.copy()
    for col in df_clean.select_dtypes(include='object').columns:
        df_clean[col] = df_clean[col].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
    return df_clean

def remove_outliers(df):
    df_clean = df.copy()
    numeric_cols = df_clean.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    return df_clean


In [7]:
def auto_clean(df):
    print("📌 Original shape:", df.shape)
    
    df = remove_duplicates(df)
    print("✅ Duplicates removed. New shape:", df.shape)

    df = handle_missing(df)
    print("✅ Missing values handled.")

    df = clean_text_columns(df)
    print("✅ Text columns standardized.")

    df = remove_outliers(df)
    print("✅ Outliers removed. Final shape:", df.shape)

    return df


In [8]:
clean_df = auto_clean(df)
clean_df.to_csv("cleaned_data.csv", index=False)


📌 Original shape: (421, 12)
✅ Duplicates removed. New shape: (421, 12)
✅ Missing values handled.
✅ Text columns standardized.
✅ Outliers removed. Final shape: (287, 12)


In [9]:
def generate_report(original_df, cleaned_df):
    print("📊 Summary Report")
    print("Original Shape:", original_df.shape)
    print("Cleaned Shape:", cleaned_df.shape)
    print("\nMissing Values Before:\n", original_df.isnull().sum())
    print("\nMissing Values After:\n", cleaned_df.isnull().sum())
    print("\nDuplicates Removed:", original_df.duplicated().sum())

generate_report(df, clean_df)


📊 Summary Report
Original Shape: (421, 12)
Cleaned Shape: (287, 12)

Missing Values Before:
 Code                0
Employee Name       0
Group               0
Item Code           0
Item Group Name     0
Category            0
Brand              36
Item Name           0
Qty                 0
Price               0
Total               0
I_Group             0
dtype: int64

Missing Values After:
 Code               0
Employee Name      0
Group              0
Item Code          0
Item Group Name    0
Category           0
Brand              0
Item Name          0
Qty                0
Price              0
Total              0
I_Group            0
dtype: int64

Duplicates Removed: 0
